# EDA y Validación - Metodología Progol

Este notebook permite explorar y validar los resultados del pipeline de generación de quinielas.

In [ ]:
# Imports necesarios
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import json
import warnings
warnings.filterwarnings('ignore')

# Configuración de visualización
plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

# Path setup
import sys
sys.path.append('../')
from src.utils.config import JORNADA_ID

print(f"Analizando Jornada: {JORNADA_ID}")

## 1. Carga de Datos

In [ ]:
# Cargar todos los archivos relevantes
try:
    # Features
    df_features = pd.read_feather(f"../data/processed/match_features_{JORNADA_ID}.feather")
    print(f"✓ Features cargadas: {df_features.shape}")
    
    # Probabilidades
    df_prob_raw = pd.read_csv(f"../data/processed/odds_norm_{JORNADA_ID}.csv")
    df_prob_blend = pd.read_csv(f"../data/processed/prob_blend_{JORNADA_ID}.csv")
    df_prob_final = pd.read_csv(f"../data/processed/prob_draw_adjusted_{JORNADA_ID}.csv")
    print("✓ Probabilidades cargadas")
    
    # Portafolio
    df_portfolio = pd.read_csv(f"../data/processed/portfolio_final_{JORNADA_ID}.csv")
    print(f"✓ Portafolio cargado: {len(df_portfolio)} quinielas")
    
    # Simulación
    df_sim = pd.read_csv(f"../data/processed/simulation_metrics_{JORNADA_ID}.csv")
    print("✓ Métricas de simulación cargadas")
    
    # Tags
    df_tags = pd.read_csv(f"../data/processed/match_tags_{JORNADA_ID}.csv")
    print("✓ Etiquetas de partidos cargadas")
    
except Exception as e:
    print(f"❌ Error cargando archivos: {e}")
    print("Verifica que hayas ejecutado los DAGs primero")

## 2. Análisis de Features

In [ ]:
# Resumen de features
print("=== RESUMEN DE FEATURES ===")
print(f"\nColumnas disponibles ({len(df_features.columns)}):")
print(list(df_features.columns))

# Estadísticas descriptivas de variables clave
key_vars = ['delta_forma', 'h2h_ratio', 'inj_weight', 'elo_diff']
available_vars = [v for v in key_vars if v in df_features.columns]

if available_vars:
    print("\n=== ESTADÍSTICAS DE VARIABLES CLAVE ===")
    display(df_features[available_vars].describe())

# Valores faltantes
print("\n=== VALORES FALTANTES ===")
missing = df_features.isnull().sum()
missing_pct = (missing / len(df_features) * 100).round(2)
missing_df = pd.DataFrame({'Faltantes': missing, 'Porcentaje': missing_pct})
display(missing_df[missing_df['Faltantes'] > 0].sort_values('Porcentaje', ascending=False))

## 3. Evolución de Probabilidades

In [ ]:
# Comparar evolución de probabilidades
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle('Evolución de Probabilidades: Raw → Blend → Final', fontsize=16)

# Preparar datos
prob_comparison = pd.DataFrame({
    'match': range(1, 15),
    'raw_L': df_features['p_raw_L'].values,
    'blend_L': df_prob_blend['p_blend_L'].values,
    'final_L': df_prob_final['p_final_L'].values,
    'raw_E': df_features['p_raw_E'].values,
    'blend_E': df_prob_blend['p_blend_E'].values,
    'final_E': df_prob_final['p_final_E'].values,
})

# Plot para Local
ax1 = axes[0, 0]
ax1.plot(prob_comparison['match'], prob_comparison['raw_L'], 'o-', label='Raw', alpha=0.7)
ax1.plot(prob_comparison['match'], prob_comparison['blend_L'], 's-', label='Blend', alpha=0.7)
ax1.plot(prob_comparison['match'], prob_comparison['final_L'], '^-', label='Final', alpha=0.7)
ax1.set_title('Probabilidad Local (L)')
ax1.set_xlabel('Partido')
ax1.set_ylabel('Probabilidad')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Plot para Empate
ax2 = axes[0, 1]
ax2.plot(prob_comparison['match'], prob_comparison['raw_E'], 'o-', label='Raw', alpha=0.7)
ax2.plot(prob_comparison['match'], prob_comparison['blend_E'], 's-', label='Blend', alpha=0.7)
ax2.plot(prob_comparison['match'], prob_comparison['final_E'], '^-', label='Final', alpha=0.7)
ax2.set_title('Probabilidad Empate (E)')
ax2.set_xlabel('Partido')
ax2.set_ylabel('Probabilidad')
ax2.legend()
ax2.grid(True, alpha=0.3)

# Diferencias Raw vs Final
ax3 = axes[1, 0]
diff_L = prob_comparison['final_L'] - prob_comparison['raw_L']
diff_E = prob_comparison['final_E'] - prob_comparison['raw_E']
diff_V = 1 - prob_comparison['final_L'] - prob_comparison['final_E'] - (1 - prob_comparison['raw_L'] - prob_comparison['raw_E'])

x = np.arange(1, 15)
width = 0.25
ax3.bar(x - width, diff_L, width, label='Local', alpha=0.7)
ax3.bar(x, diff_E, width, label='Empate', alpha=0.7)
ax3.bar(x + width, diff_V, width, label='Visitante', alpha=0.7)
ax3.set_title('Diferencia Final - Raw')
ax3.set_xlabel('Partido')
ax3.set_ylabel('Δ Probabilidad')
ax3.legend()
ax3.grid(True, alpha=0.3)
ax3.axhline(0, color='black', linewidth=0.5)

# Distribución de probabilidad máxima
ax4 = axes[1, 1]
p_max = df_prob_final[['p_final_L', 'p_final_E', 'p_final_V']].max(axis=1)
ax4.hist(p_max, bins=15, alpha=0.7, edgecolor='black')
ax4.axvline(0.60, color='red', linestyle='--', label='Umbral Ancla (0.60)')
ax4.set_title('Distribución de Probabilidad Máxima')
ax4.set_xlabel('P_max')
ax4.set_ylabel('Frecuencia')
ax4.legend()
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Resumen de cambios
print("\n=== IMPACTO DEL MODELO ===")
print(f"Empates esperados (raw): {df_features['p_raw_E'].sum():.2f}")
print(f"Empates esperados (final): {df_prob_final['p_final_E'].sum():.2f}")
print(f"Diferencia: {df_prob_final['p_final_E'].sum() - df_features['p_raw_E'].sum():.2f}")

## 4. Análisis de Etiquetas de Partidos

In [ ]:
# Análisis de etiquetas
print("=== DISTRIBUCIÓN DE ETIQUETAS ===")
etiquetas_count = df_tags['etiqueta'].value_counts()
print(etiquetas_count)
print(f"\nTotal partidos: {len(df_tags)}")

# Visualización
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Pie chart de etiquetas
colors = ['#2ecc71', '#3498db', '#e74c3c', '#95a5a6']
ax1.pie(etiquetas_count.values, labels=etiquetas_count.index, autopct='%1.1f%%', 
        colors=colors, startangle=90)
ax1.set_title('Distribución de Etiquetas de Partidos')

# Probabilidades por etiqueta
df_tags_prob = df_tags.merge(df_prob_final, on='match_id')
etiquetas = df_tags_prob['etiqueta'].unique()

positions = np.arange(len(etiquetas))
width = 0.25

for i, col in enumerate(['p_final_L', 'p_final_E', 'p_final_V']):
    means = [df_tags_prob[df_tags_prob['etiqueta'] == e][col].mean() for e in etiquetas]
    ax2.bar(positions + i*width - width, means, width, label=col[-1], alpha=0.7)

ax2.set_xlabel('Etiqueta')
ax2.set_ylabel('Probabilidad Promedio')
ax2.set_title('Probabilidades Promedio por Etiqueta')
ax2.set_xticks(positions)
ax2.set_xticklabels(etiquetas)
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Partidos específicos por etiqueta
print("\n=== DETALLE DE PARTIDOS POR ETIQUETA ===")
for etiqueta in etiquetas:
    partidos = df_tags_prob[df_tags_prob['etiqueta'] == etiqueta]
    print(f"\n{etiqueta} ({len(partidos)} partidos):")
    if 'home' in df_features.columns and 'away' in df_features.columns:
        for _, p in partidos.iterrows():
            match_info = df_features[df_features['match_id'] == p['match_id']].iloc[0]
            print(f"  Partido {p['match_id']}: {match_info['home']} vs {match_info['away']} - P_max: {p['p_max']:.3f}")

## 5. Análisis del Portafolio

In [ ]:
# Análisis del portafolio
print("=== ANÁLISIS DEL PORTAFOLIO ===")
print(f"Total de quinielas: {len(df_portfolio)}")

# Distribución de signos global
quinielas_matrix = df_portfolio.drop(columns='quiniela_id').values
total_L = (quinielas_matrix == 'L').sum()
total_E = (quinielas_matrix == 'E').sum()
total_V = (quinielas_matrix == 'V').sum()
total = total_L + total_E + total_V

print(f"\nDistribución global:")
print(f"  Local (L): {total_L} ({total_L/total:.1%})")
print(f"  Empate (E): {total_E} ({total_E/total:.1%})")
print(f"  Visitante (V): {total_V} ({total_V/total:.1%})")

# Empates por quiniela
empates_por_quiniela = []
for _, row in df_portfolio.iterrows():
    q = row.drop('quiniela_id').tolist()
    empates_por_quiniela.append(q.count('E'))

print(f"\nEmpates por quiniela:")
print(f"  Mínimo: {min(empates_por_quiniela)}")
print(f"  Máximo: {max(empates_por_quiniela)}")
print(f"  Promedio: {np.mean(empates_por_quiniela):.2f}")

# Visualización
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle('Análisis del Portafolio de Quinielas', fontsize=16)

# Heatmap de quinielas
ax1 = axes[0, 0]
# Convertir a numérico para visualización
quinielas_num = np.zeros_like(quinielas_matrix, dtype=int)
for i in range(quinielas_matrix.shape[0]):
    for j in range(quinielas_matrix.shape[1]):
        if quinielas_matrix[i,j] == 'L': quinielas_num[i,j] = 1
        elif quinielas_matrix[i,j] == 'E': quinielas_num[i,j] = 0
        else: quinielas_num[i,j] = -1

im = ax1.imshow(quinielas_num[:10], cmap='RdYlBu', aspect='auto', vmin=-1, vmax=1)
ax1.set_title('Primeras 10 Quinielas (L=Azul, E=Blanco, V=Rojo)')
ax1.set_xlabel('Partido')
ax1.set_ylabel('Quiniela')
ax1.set_xticks(range(14))
ax1.set_xticklabels(range(1, 15))

# Cobertura por partido
ax2 = axes[0, 1]
cobertura = []
for j in range(14):
    signos_partido = quinielas_matrix[:, j]
    unique_signos = len(set(signos_partido))
    cobertura.append(unique_signos)

ax2.bar(range(1, 15), cobertura, alpha=0.7)
ax2.axhline(3, color='green', linestyle='--', label='Cobertura máxima')
ax2.axhline(2, color='orange', linestyle='--', label='Cobertura media')
ax2.set_title('Cobertura de Signos por Partido')
ax2.set_xlabel('Partido')
ax2.set_ylabel('Signos Distintos Cubiertos')
ax2.legend()
ax2.grid(True, alpha=0.3)

# Distribución de empates
ax3 = axes[1, 0]
ax3.hist(empates_por_quiniela, bins=range(3, 8), alpha=0.7, edgecolor='black')
ax3.axvline(4, color='red', linestyle='--', label='Mínimo (4)')
ax3.axvline(6, color='red', linestyle='--', label='Máximo (6)')
ax3.set_title('Distribución de Empates por Quiniela')
ax3.set_xlabel('Número de Empates')
ax3.set_ylabel('Frecuencia')
ax3.legend()
ax3.grid(True, alpha=0.3)

# Correlación entre quinielas
ax4 = axes[1, 1]
# Calcular matriz de similitud
n_quinielas = min(20, len(df_portfolio))  # Limitar para visualización
similarity_matrix = np.zeros((n_quinielas, n_quinielas))

for i in range(n_quinielas):
    for j in range(n_quinielas):
        q1 = df_portfolio.iloc[i].drop('quiniela_id').tolist()
        q2 = df_portfolio.iloc[j].drop('quiniela_id').tolist()
        similarity = sum([1 for k in range(14) if q1[k] == q2[k]]) / 14
        similarity_matrix[i, j] = similarity

im = ax4.imshow(similarity_matrix, cmap='viridis', aspect='auto')
ax4.set_title(f'Matriz de Similitud (Primeras {n_quinielas} quinielas)')
ax4.set_xlabel('Quiniela')
ax4.set_ylabel('Quiniela')
cbar = plt.colorbar(im, ax=ax4)
cbar.set_label('Similitud')

plt.tight_layout()
plt.show()

## 6. Métricas de Simulación

In [ ]:
# Análisis de simulación
print("=== MÉTRICAS DE SIMULACIÓN ===")
print(f"\nEstadísticas agregadas del portafolio:")
print(f"  Pr[≥11]: {df_sim['pr_11'].mean():.2%} (±{df_sim['pr_11'].std():.2%})")
print(f"  Pr[≥10]: {df_sim['pr_10'].mean():.2%} (±{df_sim['pr_10'].std():.2%})")
print(f"  μ hits: {df_sim['mu'].mean():.2f} (±{df_sim['mu'].std():.2f})")
print(f"  σ hits: {df_sim['sigma'].mean():.2f}")

# ROI estimado
from src.utils.config import COSTO_BOLETO, PREMIO_CAT2
costo_total = len(df_portfolio) * COSTO_BOLETO
ganancia_esperada = df_sim['pr_11'].mean() * PREMIO_CAT2
roi = (ganancia_esperada / costo_total - 1) * 100

print(f"\nAnálisis económico:")
print(f"  Costo total: ${costo_total:,.0f}")
print(f"  Ganancia esperada: ${ganancia_esperada:,.0f}")
print(f"  ROI esperado: {roi:.1f}%")

# Visualización
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle('Análisis de Simulación Monte Carlo', fontsize=16)

# Distribución de Pr[≥11]
ax1 = axes[0, 0]
ax1.hist(df_sim['pr_11'], bins=20, alpha=0.7, edgecolor='black')
ax1.axvline(df_sim['pr_11'].mean(), color='red', linestyle='--', 
            label=f'Media: {df_sim["pr_11"].mean():.2%}')
ax1.set_title('Distribución de Pr[≥11] por Quiniela')
ax1.set_xlabel('Pr[≥11]')
ax1.set_ylabel('Frecuencia')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Top 10 quinielas
ax2 = axes[0, 1]
top10 = df_sim.nlargest(10, 'pr_11')
ax2.barh(range(10), top10['pr_11'], alpha=0.7)
ax2.set_yticks(range(10))
ax2.set_yticklabels(top10['quiniela_id'])
ax2.set_title('Top 10 Quinielas por Pr[≥11]')
ax2.set_xlabel('Pr[≥11]')
ax2.grid(True, alpha=0.3)

# Relación μ vs σ
ax3 = axes[1, 0]
scatter = ax3.scatter(df_sim['mu'], df_sim['sigma'], 
                      c=df_sim['pr_11'], cmap='viridis', alpha=0.6)
ax3.set_title('Relación μ vs σ (color = Pr[≥11])')
ax3.set_xlabel('μ hits')
ax3.set_ylabel('σ hits')
cbar = plt.colorbar(scatter, ax=ax3)
cbar.set_label('Pr[≥11]')
ax3.grid(True, alpha=0.3)

# Curva de probabilidad acumulada
ax4 = axes[1, 1]
hits_range = range(0, 15)
pr_at_least = []

for h in hits_range:
    # Aproximación usando μ y σ promedio
    mu_avg = df_sim['mu'].mean()
    sigma_avg = df_sim['sigma'].mean()
    pr = 1 - stats.norm.cdf(h - 0.5, mu_avg, sigma_avg)
    pr_at_least.append(pr)

ax4.plot(hits_range, pr_at_least, 'o-', linewidth=2)
ax4.axvline(11, color='red', linestyle='--', label='Objetivo (11)')
ax4.axhline(0.10, color='green', linestyle='--', label='10%')
ax4.set_title('Probabilidad Acumulada de Aciertos')
ax4.set_xlabel('Número de Aciertos')
ax4.set_ylabel('Pr[≥X]')
ax4.legend()
ax4.grid(True, alpha=0.3)
ax4.set_ylim(0, 1)

plt.tight_layout()
plt.show()

## 7. Validación de Reglas de Negocio

In [ ]:
# Validación de reglas
print("=== VALIDACIÓN DE REGLAS DE NEGOCIO ===")

validaciones = []

# 1. Distribución L-E-V global
pct_L = total_L / total
pct_E = total_E / total
pct_V = total_V / total

validaciones.append({
    'Regla': 'Distribución L (35-41%)',
    'Valor': f'{pct_L:.1%}',
    'Cumple': '✓' if 0.35 <= pct_L <= 0.41 else '✗'
})

validaciones.append({
    'Regla': 'Distribución E (25-33%)',
    'Valor': f'{pct_E:.1%}',
    'Cumple': '✓' if 0.25 <= pct_E <= 0.33 else '✗'
})

validaciones.append({
    'Regla': 'Distribución V (30-36%)',
    'Valor': f'{pct_V:.1%}',
    'Cumple': '✓' if 0.30 <= pct_V <= 0.36 else '✗'
})

# 2. Empates por quiniela
empates_ok = all(4 <= e <= 6 for e in empates_por_quiniela)
validaciones.append({
    'Regla': 'Empates por quiniela (4-6)',
    'Valor': f'{min(empates_por_quiniela)}-{max(empates_por_quiniela)}',
    'Cumple': '✓' if empates_ok else '✗'
})

# 3. Quinielas únicas
quinielas_str = df_portfolio.drop(columns='quiniela_id').apply(lambda x: ''.join(x), axis=1)
n_unique = quinielas_str.nunique()
validaciones.append({
    'Regla': 'Todas las quinielas únicas',
    'Valor': f'{n_unique}/{len(df_portfolio)}',
    'Cumple': '✓' if n_unique == len(df_portfolio) else '✗'
})

# 4. Pr[≥11] objetivo
pr11_avg = df_sim['pr_11'].mean()
validaciones.append({
    'Regla': 'Pr[≥11] ≥ 10%',
    'Valor': f'{pr11_avg:.2%}',
    'Cumple': '✓' if pr11_avg >= 0.10 else '✗'
})

# 5. Varianza reducida
sigma_avg = df_sim['sigma'].mean()
validaciones.append({
    'Regla': 'σ hits < 1.10',
    'Valor': f'{sigma_avg:.2f}',
    'Cumple': '✓' if sigma_avg < 1.10 else '✗'
})

# Mostrar tabla de validaciones
df_validaciones = pd.DataFrame(validaciones)
display(df_validaciones)

# Resumen
n_cumple = (df_validaciones['Cumple'] == '✓').sum()
n_total = len(df_validaciones)
print(f"\nRESUMEN: {n_cumple}/{n_total} reglas cumplidas")

if n_cumple == n_total:
    print("✅ TODAS LAS VALIDACIONES PASARON")
else:
    print("⚠️ ALGUNAS VALIDACIONES FALLARON - Revisar")

## 8. Comparación con Benchmarks

In [ ]:
# Comparación con benchmarks (si existen datos históricos)
print("=== COMPARACIÓN CON BENCHMARKS ===")

# Benchmarks de la metodología
benchmarks = {
    'Market-Only': {'mu': 8.11, 'sigma': 1.25, 'pr_11': 0.071, 'roi': -0.19},
    'Poisson-Only': {'mu': 8.28, 'sigma': 1.20, 'pr_11': 0.079, 'roi': -0.17},
    'Interno 2024': {'mu': 8.23, 'sigma': 1.14, 'pr_11': 0.081, 'roi': -0.12},
    'Metodología Actual': {
        'mu': df_sim['mu'].mean(),
        'sigma': df_sim['sigma'].mean(),
        'pr_11': df_sim['pr_11'].mean(),
        'roi': roi / 100
    }
}

# Crear DataFrame de comparación
df_bench = pd.DataFrame(benchmarks).T
df_bench = df_bench.round(3)

# Visualización
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle('Comparación con Benchmarks', fontsize=16)

metrics = ['mu', 'sigma', 'pr_11', 'roi']
titles = ['μ hits', 'σ hits', 'Pr[≥11]', 'ROI']

for i, (metric, title) in enumerate(zip(metrics, titles)):
    ax = axes[i//2, i%2]
    
    bars = ax.bar(df_bench.index, df_bench[metric], alpha=0.7)
    
    # Colorear la barra del método actual
    bars[-1].set_color('#2ecc71')
    
    # Agregar línea de referencia
    if metric == 'pr_11':
        ax.axhline(0.10, color='red', linestyle='--', label='Objetivo 10%')
    elif metric == 'roi':
        ax.axhline(0, color='red', linestyle='--', label='Break-even')
    
    ax.set_title(f'{title}')
    ax.set_ylabel(title)
    ax.grid(True, alpha=0.3)
    
    # Rotar etiquetas
    ax.set_xticklabels(df_bench.index, rotation=45, ha='right')
    
    if ax.get_legend_handles_labels()[0]:
        ax.legend()

plt.tight_layout()
plt.show()

# Tabla de mejoras
print("\n=== MEJORAS vs BENCHMARKS ===")
baseline = 'Interno 2024'
actual = 'Metodología Actual'

mejoras = []
for metric in metrics:
    valor_base = benchmarks[baseline][metric]
    valor_actual = benchmarks[actual][metric]
    
    if metric == 'sigma':  # Para sigma, menor es mejor
        mejora_pct = (valor_base - valor_actual) / valor_base * 100
    else:
        mejora_pct = (valor_actual - valor_base) / abs(valor_base) * 100
    
    mejoras.append({
        'Métrica': metric,
        f'{baseline}': valor_base,
        f'{actual}': valor_actual,
        'Mejora %': f'{mejora_pct:+.1f}%'
    })

df_mejoras = pd.DataFrame(mejoras)
display(df_mejoras)

print("\n💡 CONCLUSIÓN:")
if roi > 0:
    print(f"✅ El método actual logra ROI POSITIVO ({roi:.1f}%) por primera vez")
else:
    print(f"⚠️ ROI aún negativo ({roi:.1f}%), pero significativamente mejorado")

print(f"✅ Pr[≥11] aumentó de {benchmarks[baseline]['pr_11']:.1%} a {pr11_avg:.1%}")
print(f"✅ Varianza reducida de {benchmarks[baseline]['sigma']:.2f} a {sigma_avg:.2f}")

## 9. Exportación de Resultados

In [ ]:
# Guardar resultados del análisis
import os
from datetime import datetime

# Crear directorio de análisis si no existe
analysis_dir = f"../data/analysis/jornada_{JORNADA_ID}"
os.makedirs(analysis_dir, exist_ok=True)

# Guardar métricas clave
analisis_resumen = {
    'jornada': JORNADA_ID,
    'fecha_analisis': str(datetime.now()),
    'metricas': {
        'pr_11': float(pr11_avg),
        'pr_10': float(df_sim['pr_10'].mean()),
        'mu_hits': float(df_sim['mu'].mean()),
        'sigma_hits': float(sigma_avg),
        'roi_esperado': float(roi)
    },
    'distribucion': {
        'L': int(total_L),
        'E': int(total_E),
        'V': int(total_V),
        'pct_L': float(pct_L),
        'pct_E': float(pct_E),
        'pct_V': float(pct_V)
    },
    'validaciones': {
        'total': int(n_total),
        'cumplidas': int(n_cumple),
        'todas_ok': bool(n_cumple == n_total)
    }
}

# Guardar JSON
with open(f"{analysis_dir}/analisis_resumen.json", 'w') as f:
    json.dump(analisis_resumen, f, indent=2)

print(f"✅ Análisis guardado en: {analysis_dir}/analisis_resumen.json")

# Guardar versión del notebook
print("\n📊 Para guardar este notebook con los resultados:")
print("   File → Save and Export Notebook As → HTML")
print(f"   Guardar como: {analysis_dir}/analisis_completo.html")